In [16]:
import os

In [17]:
import pandas as pd

In [18]:
df = pd.read_csv("~/library/gapminder.csv")
df.head()

,population,fertility,HIV,CO2,BMI_male,GDP,BMI_female,life,child_mortality,Region
0,34811059.0,2.73,0.1,3.328945,24.59620,12314.0,129.9049,75.3,29.5,Middle East & North Africa
1,19842251.0,6.43,2.0,1.474353,22.25083,7103.0,130.1247,58.3,192.0,Sub-Saharan Africa
2,40381860.0,2.24,0.5,4.785170,27.50170,14646.0,118.8915,75.5,15.4,America
3,2975029.0,1.40,0.1,1.804106,25.35542,7383.0,132.8108,72.5,20.0,Europe & Central Asia
4,21370348.0,1.96,0.1,18.016313,27.56373,41312.0,117.3755,81.5,5.2,East Asia & Pacific


In [19]:
df.dtypes

population         float64
fertility          float64
HIV                float64
CO2                float64
BMI_male           float64
GDP                float64
BMI_female         float64
life               float64
child_mortality    float64
Region              object
dtype: object

In [20]:
import numpy as np

In [21]:
features_by_filter_method=["HIV","GDP","BMI_female","child_mortality"]

In [22]:
x=df.drop(["life"],1)
y=df["life"] 

In [28]:
numerical=x.select_dtypes(include=np.number)
categorical=x.select_dtypes(exclude=np.number)
categorical=pd.get_dummies(categorical)

In [29]:
categorical.head()

,Region_America,Region_East Asia & Pacific,Region_Europe & Central Asia,Region_Middle East & North Africa,Region_South Asia,Region_Sub-Saharan Africa
0,0,0,0,1,0,0
1,0,0,0,0,0,1
2,1,0,0,0,0,0
3,0,0,1,0,0,0
4,0,1,0,0,0,0


In [12]:
#annot=False will givw only the colours not the values
import seaborn as sns
sns.heatmap(numerical_df.corr(),annot=True)

In [30]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
#scaler.fit_transform() gives output as numpy array so store the column names before hand to prevent loss
col_names=list(numerical)
numerical=scaler.fit_transform(numerical)

In [32]:
numerical

array([[0.02885524, 0.22979398, 0.00154799, ..., 0.0934432 , 0.69160158,
        0.14157422],
       [0.01634781, 0.81616482, 0.0750774 , ..., 0.05191731, 0.70373416,
        1.        ],
       [0.03351002, 0.15213946, 0.01702786, ..., 0.11202665, 0.08368062,
        0.06708928],
       ...,
       [0.07211944, 0.09191759, 0.01315789, ..., 0.02786721, 0.25177049,
        0.12414157],
       [0.01072639, 0.72900158, 0.52399381, ..., 0.01953175, 0.83204813,
        0.48705758],
       [0.01104464, 0.40729002, 0.58204334, ..., 0.00556228, 0.80583998,
        0.50501849]])

In [36]:
numerical=pd.DataFrame(numerical)
numerical.columns=col_names
x=pd.concat([numerical,categorical],1)

In [37]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
knn=KNeighborsRegressor(n_neighbors=5)
cross_val_score(knn,x,y,cv=10, scoring="r2")

array([0.73669977, 0.74702889, 0.62634816, 0.69099834, 0.6544671 ,
       0.81120102, 0.7446377 , 0.76684161, 0.90161633, 0.86892009])

In [38]:
params={"n_neighbors":np.arange(1,10)}
params
knn=KNeighborsRegressor()
knn_cv=GridSearchCV(knn,param_grid=params,cv=10,scoring="r2")

In [39]:
knn_cv.fit(x,y)

GridSearchCV(cv=10, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': array([1, 2, 3, 4, 5, 6, 7, 8, 9])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=0)

In [40]:
knn_cv.best_estimator_

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=4, p=2,
          weights='uniform')

In [41]:
knn_cv.best_score_

0.7574880832829246

In [48]:
from sklearn.model_selection import train_test_split as tts
x_train,x_test,y_train,y_test = tts(x,y,test_size=0.2,random_state=42)
knn.fit(x_train,y_train)
y_pred=knn.predict(x_test)
mean_squared_error(y_test,y_pred)
np.sqrt(mean_squared_error(y_test,y_pred))

2.89163819115937

In [47]:
r2_score(y_test,y_pred)

0.8748111895269864

In [43]:
x=x[features_by_filter_method]
x.head()

,HIV,GDP,BMI_female,child_mortality
0,0.001548,0.093443,0.691602,0.141574
1,0.075077,0.051917,0.703734,1.000000
2,0.017028,0.112027,0.083681,0.067089
3,0.001548,0.054149,0.852002,0.091389
4,0.001548,0.324525,0.000000,0.013207


In [81]:
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
x=df.drop(["life"],1)
y=df["life"]
x=pd.get_dummies(x)
scaler=MinMaxScaler()
col_names=list(x)
x=scaler.fit_transform(x)
x_train,x_test,y_train,y_test = tts(x,y,test_size=0.2,random_state=42)
x=pd.DataFrame(x)
x.columns=col_names
x_train,x_test,y_train,y_test = tts(x,y,test_size=0.2,random_state=42)

In [84]:
lin_reg=LinearRegression()
lin_reg

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [86]:
lin_reg.fit(x_train,y_train)
y_pred=lin_reg.predict(x_test)
r2_score(y_test,y_pred)
np.sqrt(mean_squared_error(y_test,y_pred))

2.346972417900388

In [88]:
cross_val_score(lin_reg,x,y,cv=10)

array([0.7167508 , 0.80157728, 0.81668174, 0.82942625, 0.7418073 ,
       0.93913741, 0.83464389, 0.80949561, 0.95082034, 0.94307859])

In [90]:
ridge= RidgeCV(alphas=np.arange(0.1,2,0.1))
ridge.fit(x,y)

RidgeCV(alphas=array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9]),
    cv=None, fit_intercept=True, gcv_mode=None, normalize=False,
    scoring=None, store_cv_values=False)

In [94]:
#model of better fit
#alpha_ gives the best penalty parameter(lamda)
ridge.alpha_

0.2

In [96]:
ridge.score(x,y)

0.9031834826223613

In [97]:
lasso=LassoCV(alphas=np.arange(0.1,2,0.1))

In [98]:
lasso.fit(x,y)

LassoCV(alphas=array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9]),
    copy_X=True, cv=None, eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

In [99]:
lasso.score(x,y)

0.8825658083305495